# ANOVA
Process Image.csv files, which has one measurement per WSI patch.

Sources of variance (ignoring the WSI level):
* Total = between patches = sum over patches (patch deviation from global mean, sq)
* Between class = weight sum over class (sum class mean deviation from global mean, sq)
* Within class = sum over class (sum class mean deviation from global mean, sq)
* Between patient = weight sum over patients (sum patient mean deviation from global mean, sq) 
* Within patient = sum over patients (sum patch deviation from patient mean, sq)
* Residual = Total - explained

In [1]:
import numpy as np
import pandas as pd
import os
import csv
from datetime import datetime
print(datetime.now())

2022-07-25 09:09:48.686226


In [14]:
BASE_PATH_IN='/home/jrm/Adjeroh/Naved/CP_80K/'
BASE_PATH_IN='/Users/jasonmiller/Downloads/CellProfilerFiltered/'
CLASS_PATH_IN='/Users/jasonmiller/Downloads/TrainTestSplit/'
CSVFILE='Process100_Image.csv'
MEASUREMENT='Mean_Nucleus_AreaShape_MeanRadius'
MCOLUMN = 888  # zero based column number within csv

In [3]:
def validate_column():
    print('Validating column',MCOLUMN,'named',MEASUREMENT,':')
    for patient in os.listdir(BASE_PATH_IN):
        if patient.startswith('TCGA-'):
            print('.', end='')
            file = BASE_PATH_IN + patient + '/' + CSVFILE
            # print(file)
            with open(file) as infile:
                reader = csv.reader(infile)
                rows = 0
                for row in reader:
                    value = row[MCOLUMN]
                    if rows == 0:
                        if value!=MEASUREMENT:
                            raise Exception('For '+patient+', column name is '+value)
                    else:
                        numeric = float(value)  # throws exception if not numeric
                    rows += 1
                if rows < 2:
                    raise Exception('no data rows for '+patient)
    print('\nColumn validated!')
    
if False:  # no need to run this again on same dataset
    validate_column()

In [4]:
def count_one_patient(directory):
    file = BASE_PATH_IN + directory + '/' + CSVFILE
    with open(file) as infile:
        total = 0
        count = 0
        reader = csv.reader(infile)
        header = True
        for row in reader:
            value = row[MCOLUMN]
            if header:
                header = False
            else:
                total += float(value)
                count += 1
    return count, total

In [10]:
def get_total_variance(global_mean):
    total_SSE = 0
    for patient in os.listdir(BASE_PATH_IN):
        if patient.startswith('TCGA-'):
            file = BASE_PATH_IN + patient + '/' + CSVFILE
            with open(file) as infile:
                reader = csv.reader(infile)
                header = True
                for row in reader:
                    value = row[MCOLUMN]
                    if header:
                        header = False
                    else:
                        value = float(value)
                        error = global_mean - value
                        square = error**2
                        total_SSE += square
    return total_SSE

In [7]:
def get_global_count():
    grand_count = 0
    grand_total = 0
    for patient in os.listdir(BASE_PATH_IN):
        if patient.startswith('TCGA-'):
            patient_count, patient_total = count_one_patient(patient)
            grand_count += patient_count
            grand_total += patient_total
            patient_mean = patient_total / patient_count
            # print(patient,patient_mean)
    return grand_count, grand_total


In [8]:
global_count, global_total = get_global_count()
global_mean = global_total / global_count
print('Global mean',global_mean)

TCGA-02-0010-01Z-00 2.9064318134506393
TCGA-HT-7616-01Z-00 3.704587956514372
TCGA-14-1452-01Z-00 2.9461776766095933
TCGA-28-1746-01Z-00 3.2849103796962877
TCGA-DU-6404-01Z-00 3.6864243564491557
TCGA-S9-A6WE-01Z-00 3.3019203550651817
TCGA-S9-A6WI-01Z-00 3.376396004883294
TCGA-QH-A6CZ-01Z-00 3.5959338396654132
TCGA-DU-6408-01Z-00 3.514234180896044
TCGA-HT-7881-01Z-00 3.6548921233716807
TCGA-DU-7009-01Z-00 3.38643809327903
TCGA-26-5139-01Z-00 3.287579389830524
TCGA-DU-8167-01Z-00 3.746975492448168
TCGA-HW-7491-01Z-00 3.558147832054447
TCGA-14-1829-01Z-00 3.1441575435397016
TCGA-06-0129-01Z-00 3.0907424041228335
TCGA-DU-8165-01Z-00 3.878493938639796
TCGA-DU-7015-01Z-00 3.340709092089833
TCGA-06-0125-01Z-00 2.9174883245532013
TCGA-HT-7854-01Z-00 3.894404010019071
TCGA-DU-5874-01Z-00 3.6498957610084566
TCGA-QH-A6XA-01Z-00 3.4080852779569106
TCGA-HT-7606-01Z-00 3.8012705232240425
TCGA-02-0439-01Z-00 3.15328227036864
TCGA-26-5133-01Z-00 3.8956279552755624
TCGA-HT-A618-01Z-00 3.5840964414621093

In [11]:
total_SSE = get_total_variance(global_mean)
print('Total SSE',total_SSE)

Total SSE 21681.121924016566


In [15]:
# load classes
def load_patient_to_class():
    PATIENT_TO_CLASS={}
    file1 = CLASS_PATH_IN + 'fold0_train.txt'
    file2 = CLASS_PATH_IN + 'fold0_test.txt'
    for file in [file1,file2]:
        with open(file) as infile:
            reader = csv.reader(infile)
            for row in reader:
                patchfile = row[0]
                patient = patchfile[:19]
                cancer = int(row[1])
                # The file gives this information redundantly
                PATIENT_TO_CLASS[patient] = cancer
    return PATIENT_TO_CLASS
mapper = load_patient_to_class()
mapper

{'TCGA-06-0129-01Z-00': 0,
 'TCGA-14-1452-01Z-00': 0,
 'TCGA-14-1795-01Z-00': 0,
 'TCGA-06-0125-01Z-00': 0,
 'TCGA-06-0878-01Z-00': 0,
 'TCGA-27-1831-01Z-00': 0,
 'TCGA-28-1749-01Z-00': 0,
 'TCGA-14-0786-01Z-00': 0,
 'TCGA-08-0520-01Z-00': 0,
 'TCGA-06-1084-01Z-00': 0,
 'TCGA-06-0645-01Z-00': 0,
 'TCGA-26-5133-01Z-00': 0,
 'TCGA-26-5139-01Z-00': 0,
 'TCGA-15-1447-01Z-00': 0,
 'TCGA-14-1401-01Z-00': 0,
 'TCGA-08-0518-01Z-00': 0,
 'TCGA-06-0214-01Z-00': 0,
 'TCGA-14-1829-01Z-00': 0,
 'TCGA-14-1453-01Z-00': 0,
 'TCGA-06-5412-01Z-00': 0,
 'TCGA-26-5134-01Z-00': 0,
 'TCGA-06-0190-01Z-00': 0,
 'TCGA-08-0517-01Z-00': 0,
 'TCGA-06-0189-01Z-00': 0,
 'TCGA-06-0216-01Z-00': 0,
 'TCGA-41-5651-01Z-00': 0,
 'TCGA-02-0285-01Z-00': 0,
 'TCGA-14-0789-01Z-00': 0,
 'TCGA-02-0446-01Z-00': 0,
 'TCGA-14-0787-01Z-00': 0,
 'TCGA-02-0033-01Z-00': 0,
 'TCGA-06-0211-01Z-00': 0,
 'TCGA-06-0209-01Z-00': 0,
 'TCGA-15-1446-01Z-00': 0,
 'TCGA-06-0201-01Z-00': 0,
 'TCGA-06-1801-01Z-00': 0,
 'TCGA-HT-A617-01Z-00': 1,
 